# Feature Engineering Notebook

## Objectives

*   Engineer features for Classification, Regression and Cluster models


## Inputs

* inputs/datasets/cleaned/TrainSet.csv
* inputs/datasets/cleaned/TestSet.csv

## Outputs

* generate a list with variables to engineer

## Conclusions



* Feature Engineering Transformers

  


---

# Change working directory

We need to change the working directory from its current folder to its parent folder

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of current directory the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Cleaned Data

Train Set

In [ ]:
import pandas as pd
train_set_path = "outputs/datasets/cleaned/TrainSetCleaned.csv"
TrainSet = pd.read_csv(train_set_path)
TrainSet.head(3)

Test Set

In [ ]:
test_set_path = 'outputs/datasets/cleaned/TestSetCleaned.csv'
TestSet = pd.read_csv(test_set_path)
TestSet.head(3)

# Data Exploration

In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=TrainSet, minimal=True)
pandas_report.to_notebook_iframe()

# Correlation and PPS Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

def heatmap_corr(df,threshold, figsize=(20,12), font_annot = 8):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=figsize)
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df,threshold, figsize=(20,12), font_annot = 8):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=np.bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=figsize)
      ax = sns.heatmap(df, annot=True, xticklabels=True,yticklabels=True,
                       mask=mask,cmap='rocket_r', annot_kws={"size": font_annot},
                       linewidth=0.05,linecolor='grey')
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
  df_corr_spearman = df.corr(method="spearman")
  df_corr_pearson = df.corr(method="pearson")

  pps_matrix_raw = pps.matrix(df)
  pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

  pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
  print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
  print(pps_score_stats.round(3))

  return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix,CorrThreshold,PPS_Threshold,
                      figsize=(20,12), font_annot=8 ):

  print("\n")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationship \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(TrainSet)

In [ ]:
%matplotlib inline
DisplayCorrAndPPS(df_corr_pearson=df_corr_pearson,
                  df_corr_spearman=df_corr_spearman, 
                  pps_matrix=pps_matrix,
                  CorrThreshold=0.6, PPS_Threshold=0.15,
                  figsize=(10,10), font_annot=8)

# Feature Engineering

## Custom Variable

Using the variables we believe add up to the total size of the house, we will add them together to create a new variable `TotalArea` which will then be run through our feature engineering along with the rest of the data and see if it can be used later to obtain a better prediction of `SalePrice`.

In [ ]:
from feature_engine.creation import MathFeatures

transformer = MathFeatures(
    variables=[
        '1stFlrSF',
        '2ndFlrSF',
        'BsmtFinSF1',
        'BsmtUnfSF',
        'EnclosedPorch',
        'GarageArea',
        'GrLivArea',
        'LotArea',
        'MasVnrArea',
        'OpenPorchSF',
        'TotalBsmtSF',
        'WoodDeckSF'
    ],
    func=['sum'],
    new_variables_names=[
        'TotalArea'
    ]
)

transformer.fit(TrainSet)

TrainSet, TestSet = transformer.transform(TrainSet) , transformer.transform(TestSet)
TrainSet.head(3)

In [ ]:
TestSet.head(3)

## Custom Functions

We'll be using another function provided by Code Institute, it is used for quick feature engineering on numerical and categorical variables to decide which transformation can better transform the distribution shape.

In [ ]:
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from feature_engine.encoding import OrdinalEncoder
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')



def FeatureEngineeringAnalysis(df,analysis_type=None):


  """
  - used for quick feature engineering on numerical and categorical variables
  to decide which transformation can better transform the distribution shape 
  - Once transformed, use a reporting tool, like pandas-profiling, to evaluate distributions

  """
  check_missing_values(df)
  allowed_types= ['numerical', 'ordinal_encoder',  'outlier_winsorizer', 'outlier_trimmer']
  check_user_entry_on_analysis_type(analysis_type, allowed_types)
  list_column_transformers = define_list_column_transformers(analysis_type)
  
  
  # Loop in each variable and engineer the data according to the analysis type
  df_feat_eng = pd.DataFrame([])
  for column in df.columns:
    # create additional columns (column_method) to apply the methods
    df_feat_eng = pd.concat([df_feat_eng, df[column]], axis=1)
    for method in list_column_transformers:
      df_feat_eng[f"{column}_{method}"] = df[column]
      
    # Apply transformers in respectives column_transformers
    df_feat_eng,list_applied_transformers = apply_transformers(analysis_type, df_feat_eng, column)

    # For each variable, assess how the transformations perform
    transformer_evaluation(column, list_applied_transformers, analysis_type, df_feat_eng)

  return df_feat_eng


def check_user_entry_on_analysis_type(analysis_type, allowed_types):
  ### Check analyis type
  if analysis_type == None:
    raise SystemExit(f"You should pass analysis_type parameter as one of the following options: {allowed_types}")
  if analysis_type not in allowed_types:
      raise SystemExit(f"analysis_type argument should be one of these options: {allowed_types}")

def check_missing_values(df):
  if df.isna().sum().sum() != 0:
    raise SystemExit(
        f"There is missing value in your dataset. Please handle that before getting into feature engineering.")



def define_list_column_transformers(analysis_type):
  ### Set suffix colummns acording to analysis_type
  if analysis_type=='numerical':
    list_column_transformers = ["log_e","log_10","reciprocal", "power","box_cox","yeo_johnson"]
  
  elif analysis_type=='ordinal_encoder':
    list_column_transformers = ["ordinal_encoder"]

  elif analysis_type=='outlier_winsorizer':
    list_column_transformers = ['iqr']

  elif analysis_type=='outlier_trimmer':
    list_column_transformers = ['iqr']

  return list_column_transformers



def apply_transformers(analysis_type, df_feat_eng, column):


  for col in df_feat_eng.select_dtypes(include='category').columns:
    df_feat_eng[col] = df_feat_eng[col].astype('object')


  if analysis_type=='numerical':
    df_feat_eng,list_applied_transformers = FeatEngineering_Numerical(df_feat_eng,column)
  
  elif analysis_type=='outlier_winsorizer':
    df_feat_eng,list_applied_transformers = FeatEngineering_OutlierWinsorizer(df_feat_eng,column)

  elif analysis_type=='outlier_trimmer':
    df_feat_eng,list_applied_transformers = FeatEngineering_OutlierTrimmer(df_feat_eng,column)

  elif analysis_type=='ordinal_encoder':
    df_feat_eng,list_applied_transformers = FeatEngineering_CategoricalEncoder(df_feat_eng,column)

  return df_feat_eng,list_applied_transformers



def transformer_evaluation(column, list_applied_transformers, analysis_type, df_feat_eng):
  # For each variable, assess how the transformations perform
  print(f"* Variable Analyzed: {column}")
  print(f"* Applied transformation: {list_applied_transformers} \n")
  for col in [column] + list_applied_transformers:
    
    if analysis_type!='ordinal_encoder':
      DiagnosticPlots_Numerical(df_feat_eng, col)
    
    else:
      if col == column: 
        DiagnosticPlots_Categories(df_feat_eng, col)
      else:
        DiagnosticPlots_Numerical(df_feat_eng, col)

    print("\n")



def DiagnosticPlots_Categories(df_feat_eng, col):
  plt.figure(figsize=(4, 3))
  sns.countplot(data=df_feat_eng, x=col,palette=['#432371'],order = df_feat_eng[col].value_counts().index)
  plt.xticks(rotation=90) 
  plt.suptitle(f"{col}", fontsize=30,y=1.05)        
  plt.show()
  print("\n")



def DiagnosticPlots_Numerical(df, variable):
  fig, axes = plt.subplots(1, 3, figsize=(12, 4))
  sns.histplot(data=df, x=variable, kde=True,element="step",ax=axes[0]) 
  stats.probplot(df[variable], dist="norm", plot=axes[1])
  sns.boxplot(x=df[variable],ax=axes[2])
  
  axes[0].set_title('Histogram')
  axes[1].set_title('QQ Plot')
  axes[2].set_title('Boxplot')
  fig.suptitle(f"{variable}", fontsize=30,y=1.05)
  plt.tight_layout()
  plt.show()


def FeatEngineering_CategoricalEncoder(df_feat_eng,column):
  list_methods_worked = []
  try:  
    encoder= OrdinalEncoder(encoding_method='arbitrary', variables = [f"{column}_ordinal_encoder"])
    df_feat_eng = encoder.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_ordinal_encoder")
  
  except: 
    df_feat_eng.drop([f"{column}_ordinal_encoder"],axis=1,inplace=True)
    
  return df_feat_eng,list_methods_worked


def FeatEngineering_OutlierWinsorizer(df_feat_eng,column):
  list_methods_worked = []

  ### Winsorizer iqr
  try: 
    disc=Winsorizer(
        capping_method='iqr', tail='both', fold=1.5, variables = [f"{column}_iqr"])
    df_feat_eng = disc.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_iqr")
  except: 
    df_feat_eng.drop([f"{column}_iqr"],axis=1,inplace=True)


  return df_feat_eng,list_methods_worked


def FeatEngineering_OutlierTrimmer(df_feat_eng,column):
  list_methods_worked = []

  ### Trimmer iqr
  try: 
    disc=OutlierTrimmer(
        capping_method='iqr', tail='both', fold=1.5, variables = [f"{column}_iqr"])
    df_feat_eng = disc.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_iqr")
  except: 
    df_feat_eng.drop([f"{column}_iqr"],axis=1,inplace=True)


  return df_feat_eng,list_methods_worked




def FeatEngineering_Numerical(df_feat_eng,column):

  list_methods_worked = []

  ### LogTransformer base e
  try: 
    lt = vt.LogTransformer(variables = [f"{column}_log_e"])
    df_feat_eng = lt.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_log_e")
  except: 
    df_feat_eng.drop([f"{column}_log_e"],axis=1,inplace=True)

    ### LogTransformer base 10
  try: 
    lt = vt.LogTransformer(variables = [f"{column}_log_10"],base='10')
    df_feat_eng = lt.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_log_10")
  except: 
    df_feat_eng.drop([f"{column}_log_10"],axis=1,inplace=True)

  ### ReciprocalTransformer
  try:
    rt = vt.ReciprocalTransformer(variables = [f"{column}_reciprocal"])
    df_feat_eng =  rt.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_reciprocal")
  except:
    df_feat_eng.drop([f"{column}_reciprocal"],axis=1,inplace=True)

  ### PowerTransformer
  try:
    pt = vt.PowerTransformer(variables = [f"{column}_power"])
    df_feat_eng = pt.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_power")
  except:
    df_feat_eng.drop([f"{column}_power"],axis=1,inplace=True)

  ### BoxCoxTransformer
  try:
    bct = vt.BoxCoxTransformer(variables = [f"{column}_box_cox"])
    df_feat_eng = bct.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_box_cox")
  except:
    df_feat_eng.drop([f"{column}_box_cox"],axis=1,inplace=True)


  ### YeoJohnsonTransformer
  try:
    yjt = vt.YeoJohnsonTransformer(variables = [f"{column}_yeo_johnson"])
    df_feat_eng = yjt.fit_transform(df_feat_eng)
    list_methods_worked.append(f"{column}_yeo_johnson")
  except:
        df_feat_eng.drop([f"{column}_yeo_johnson"],axis=1,inplace=True)


  return df_feat_eng,list_methods_worked


In [ ]:
def plot_histogram_and_boxplot(df):
  for col in df.columns:
    fig, axes = plt.subplots(nrows=2 ,ncols=1 ,figsize=(7,7), gridspec_kw={"height_ratios": (.15, .85)})
    sns.boxplot(data=df, x=col, ax=axes[0])
    sns.histplot(data=df, x=col, kde=True, ax=axes[1])
    fig.suptitle(f"{col} Distribution - Boxplot and Histogram")
    plt.show()

    IQR = df[col].quantile(q=0.75) - df[col].quantile(q=0.25)
    print(
        f"This is the range where a datapoint is not an outlier: from "
        f"{(df[col].quantile(q=0.25) - 1.5*IQR).round(2)} to "
        f"{(df[col].quantile(q=0.75) + 1.5*IQR).round(2)}")
    print("\n")

## Feature Engineering Spreadsheet Summary

![FeatureEngineeringSpreadsheet](../media/FeatureEngineering.png)

## Dealing with Feature Engineering

### Categorical Enconding

For categorical encoding we're just looking to convert the variable names into numbers. The QQ Plot and Box plots aren't of any major value here. 

* Step 1: Select variable(s)

In [ ]:
variables_engineering= ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual']
variables_engineering

* Step 2: Create a separate dataframe, with your variable(s)

In [ ]:
df_engineering = TrainSet[variables_engineering].copy()
df_engineering.head(3)

* Step 3: Create engineered variables(s) applying the transformation(s), assess engineered variables distribution and select most suitable method for each variable

In [ ]:
df_engineering = FeatureEngineeringAnalysis(df=df_engineering, analysis_type='ordinal_encoder')

All of the variables have been converted to numbers as expected and nothing appears to be out of place, so we are happy to apply the transformations.



* Step 4 - Apply the selected transformation to the Train and Test set

In [ ]:
encoder = OrdinalEncoder(encoding_method='arbitrary', variables = variables_engineering)
TrainSet = encoder.fit_transform(TrainSet)
TestSet = encoder.transform(TestSet)

print("* Categorical encoding - ordinal transformation done!")

### Numerical Transformation

Next are the numerical transformations. Here we are looking to apply transformers that normalize the data, by making the data less skewed and more symetrical.

Many of the variables that contain zeros, ie the house doesn't have a second floor or a wood deck, will have strong negative kurtosis which we won't want to fix but we can hopefully leave that in place whilst normalizing the rest of the data.

* Step 1: Select variable(s)

In [ ]:
variables_engineering = ['1stFlrSF', 
                    '2ndFlrSF', 
                    'BsmtFinSF1', 
                    'BsmtUnfSF', 
                    'EnclosedPorch', 
                    'GarageArea', 
                    'GrLivArea', 
                    'LotArea', 
                    'LotFrontage', 
                    'MasVnrArea', 
                    'OpenPorchSF', 
                    'TotalBsmtSF', 
                    'WoodDeckSF', 
                    'TotalArea']
variables_engineering

* Step 2: Create a separate dataframe, with your variable(s)

In [ ]:
df_engineering = TrainSet[variables_engineering].copy()
df_engineering.head(3)

* Step 3: Create engineered variables(s) applying the transformation(s), assess engineered variables distribution and select most suitable method

In [ ]:
df_engineering = FeatureEngineeringAnalysis(df=df_engineering, analysis_type='numerical')

`1stFlrSF`, `GrLivArea` and `TotalArea` appear to take well to the `log_e` and `yeo_johnson` methods so for now we will use log e for these variables and trim them of the outliers. 

We don't want to use the winsorizer as from out previous studies like in notebook 2, we saw that some of the outliers have other strong outlier variables. For example, some of the largest `GrLivArea` had very low `SalePrice` in comparison to the rest of the data. It would be safer to cut these records out of the data than putting a cap on them, as they are still likely to effect the data negatively.

For `2ndFlrSF`, `BsmtFinSF1`, `BsmtUnfSF`, `GarageArea`, `MasVnrArea`, `OpenPorchSF`, `TotalBsmtSF` and `WoodDeckSF` we will use `power` transformer as it keeps the strong kurtosis that these variables carry with all their zeros whilst normalizing the rest of the data.

`EnclosedPorch` doesn't have enough data that aren't zeros to justify using the power transformer on.

And `LotArea` and `LotFrontage` whilst they do contain a small number of zeros are better suited to `log_e` and `yeo_johnson` so again we'll for now use log e.


* Step 4 - Apply the selected transformation to the Train and Test set

In [ ]:
encoder = vt.LogTransformer(
    variables = ['1stFlrSF', 
        'GrLivArea', 
        'TotalArea', 
        'LotArea', 
        'LotFrontage', ])
TrainSet = encoder.fit_transform(TrainSet)
TestSet = encoder.transform(TestSet)

print("* Categorical encoding - ordinal transformation done!")

In [ ]:
encoder = vt.PowerTransformer(
    variables = ['2ndFlrSF', 
        'BsmtFinSF1', 
        'BsmtUnfSF', 
        'GarageArea', 
        'MasVnrArea',
        'OpenPorchSF',
        'TotalBsmtSF',
        'WoodDeckSF', ])
TrainSet = encoder.fit_transform(TrainSet)
TestSet = encoder.transform(TestSet)

print("* Categorical encoding - ordinal transformation done!")

### Outlier Trimmer

* Step 1: Select variable(s)

In [ ]:
from feature_engine.outliers import OutlierTrimmer

variables_engineering = ['SalePrice', 'GrLivArea', 'TotalArea']
variables_engineering

* Step 2: Create a separate dataframe, with your variable(s)

In [ ]:
df_engineering = TrainSet[variables_engineering].copy()
df_engineering.head(3)

* Step 3: Create engineered variables(s) applying the transformation(s), assess engineered variables distribution and select most suitable method

In [ ]:
df_transformed = OutlierTrimmer(capping_method='iqr', tail='both', fold=1.5, variables=variables_engineering)
df_transformed = df_transformed.fit_transform(df_engineering)

In [ ]:
print("========= Before Transformation ========= \n")
plot_histogram_and_boxplot(TrainSet[['SalePrice', 'GrLivArea', 'TotalArea']])
print("\n\n ========= After Transformation =========")
plot_histogram_and_boxplot(df=df_transformed)

* Step 4 - Apply the selected transformation to the Train and Test set

In [ ]:
print(f"* The dataset has {len(TrainSet)} rows, considering outliers.")

transformer = OutlierTrimmer(
    variables=['SalePrice', 'GrLivArea', 'TotalArea'],
    tail = 'both',
    capping_method='iqr',
    fold=1.5
)

transformer.fit(TrainSet)

TrainSet, TestSet = transformer.transform(TrainSet) , transformer.transform(TestSet)


print(f"* Once it is transformed with OutlierTrimmer, dataset has {len(TrainSet)} rows")

### SmartCorrelatedSelection Variables

* Step 1: Select variable(s)

In [ ]:
# for this transformer, we don't need to select variables, since we need all variables for this transformer

* Step 2: Create a separate dataframe, with your variable(s)

In [ ]:
df_engineering = TrainSet.copy()
df_engineering.head(3)

* Step 3: Create engineered variables(s) applying the transformation(s)

In [ ]:
from feature_engine.selection import SmartCorrelatedSelection
corr_sel = SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.6,selection_method="variance", missing_values='ignore')

corr_sel.fit_transform(df_engineering)
corr_sel.correlated_feature_sets_

In [ ]:
corr_sel.features_to_drop_

---

# Conclusion



Feature Engineering Transformers
  * OrdinalEncoder(encoding_method='arbitrary', variables = ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])
  * vt.LogTransformer(variables = ['1stFlrSF', 'GrLivArea', 'TotalArea', 'LotArea', 'LotFrontage'])
  * vt.PowerTransformer(variables = ['2ndFlrSF', 'BsmtFinSF1', 'BsmtUnfSF', 'GarageArea', 'MasVnrArea', 'OpenPorchSF','TotalBsmtSF', 'WoodDeckSF']) 
  * OutlierTrimmer(capping_method='iqr', tail='both', fold=1.5, ['SalePrice', 'GrLivArea', 'TotalArea'])
  * SmartCorrelatedSelection - Features to drop: 